# Mesh

See [Mesh in MIKE IO Documentation](https://dhi.github.io/mikeio/mesh.html)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mikeio

## A simple mesh

Let's consider a simple mesh consisting of 2 triangular elements. 

:::{note}
Example data can be found in the `mini_book/data` folder in this [zip-file](https://github.com/DHI/book-learn-mikeio-fmskill/archive/refs/heads/main.zip).
:::


In [ ]:
fn = "data/two_elements.mesh"

In [ ]:
with open(fn, "r") as f:
    print(f.read())

In [ ]:
msh = mikeio.open(fn)
msh

In [ ]:
msh.plot(show_mesh=True);

In [ ]:
msh.node_coordinates

In [ ]:
msh.element_table

In [ ]:
msh.element_coordinates

In [ ]:
msh.get_element_area()

Let's plot the node and element coordinates:

In [ ]:
xn, yn = msh.node_coordinates[:,0], msh.node_coordinates[:,1]
xe, ye = msh.element_coordinates[:,0], msh.element_coordinates[:,1]

ax = msh.plot(show_mesh=True)
ax.plot(xn, yn, 'ro', markersize=10)
ax.plot(xe, ye, 'bx', markersize=10)

### Boundary polylines

It can sometimes be convenient to have mesh boundary as a polyline (or multiple in case of more complex meshes). 

In [ ]:
bxy = msh.boundary_polylines.exteriors[0].xy
plt.plot(bxy[:,0], bxy[:,1])
plt.axis("equal");

## Inside domain?

MIKE IO has a method for determining if a point (or a list of points) is inside the domain: 

* contains()

In [ ]:
pt_1 = [2.0, 1.2]
msh.contains(pt_1)[0]

In [ ]:
# or multiple points at the same time
pt_2 = [4.0, 1.2]
pts = np.array([pt_1, pt_2])
msh.contains(pts)

In [ ]:
plt.plot(bxy[:,0], bxy[:,1], label='boundary')
plt.plot(xe[0], ye[0], 'b*', markersize=10, label="center, elem 0")
plt.plot(xe[1], ye[1], 'c*', markersize=10, label="center, elem 1")
plt.plot(*pt_1, 'go', markersize=10, label="pt_1")
plt.plot(*pt_2, 'rs', markersize=10, label="pt_2")
plt.axis("equal")
plt.legend(loc="upper right");

## Find element containing point

MIKE IO has a method for obtaining the index of the element *containing* a point: 

* find_index()

In [ ]:
g = msh.geometry
g.find_index(coords=pt_1)[0]

MIKE IO also has a method for obtaining a list of the n *closest* element centers: 

* find_nearest_elements()

In [ ]:
g.find_nearest_elements(pt_1)

In [ ]:
g.find_nearest_elements(pt_1, return_distances=True)

In [ ]:
g.find_nearest_elements(pt_1, n_nearest=2)

In [ ]:
# for multiple points
g.find_nearest_elements(pts, return_distances=True)

## A larger mesh

A dfsu file can also be loaded as a Mesh object

In [ ]:
fn = "data/FakeLake.dfsu"
msh = mikeio.Mesh(fn)

In [ ]:
msh.plot();

In [ ]:
msh.max_nodes_per_element

In [ ]:
msh.plot_boundary_nodes();

In [ ]:
bnd = msh.boundary_polylines
bnd.n_exteriors, bnd.n_interiors

In [ ]:
ext0 = bnd.exteriors[0]
plt.plot(ext0.xy[:,0], ext0.xy[:,1], label='exterior 0')
int0 = bnd.interiors[0]
plt.plot(int0.xy[:,0], int0.xy[:,1], label='interior 0')
plt.legend();

In [ ]:
# msh.to_shapely()

## Change depth

In [ ]:
msh = mikeio.Mesh(fn)
msh.plot();

In [ ]:
# zn == nodes, not elements!
msh.zn = np.clip(msh.zn, -15, 0) # clip depth to interval [-15,0]

In [ ]:
msh.plot();

In [ ]:
msh.write('Fake_lake_clip15.mesh')   # save to a new file

See the [MIKE IO Mesh Example notebook](https://nbviewer.jupyter.org/github/DHI/mikeio/blob/main/notebooks/Mesh.ipynb) for more Mesh operations (including shapely operations).